In [1]:
import sqlite3
import pandas as pd

In [6]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [7]:
sql_salaries = '''
drop table if exists salaries;
create table salaries (emp_id integer, depname varchar(20), emp varchar(20), role varchar(20), salary integer);
insert into salaries (emp_id, depname, emp,role, salary) values (1, 'dep1','Christiano','Manager', 1000);
insert into salaries (emp_id, depname, emp,role, salary) values (2, 'dep1','Eric','Manager', 1500);
insert into salaries (emp_id, depname, emp,role, salary) values (3, 'dep1','Ryan','Director', 2000);
insert into salaries (emp_id, depname, emp,role, salary) values (4, 'dep1', 'Bryan','Director',1500);
insert into salaries (emp_id, depname, emp,role, salary) values (5, 'dep2', 'Doreen','Manager',2000);
insert into salaries (emp_id, depname, emp,role, salary) values (6, 'dep2', 'Gabriel','Manager',3000);
insert into salaries (emp_id, depname, emp,role, salary) values (7, 'dep2', 'Angel','Manager',2400);
insert into salaries (emp_id, depname, emp,role, salary) values (8, 'dep2', 'Robin','Director',2700);
insert into salaries (emp_id, depname, emp,role, salary) values (9, 'dep3','Sarah','Manager', 1900);
insert into salaries (emp_id, depname, emp,role, salary) values (10, 'dep3', 'Peter','Manager',1700);
'''

In [4]:
sql_salaries = '''
insert into salaries (emp_id, depname, emp,role, salary) values (9, 'dep3','Sarah','Manager', 1900);
insert into salaries (emp_id, depname, emp,role, salary) values (10, 'dep3', 'Peter','Manager',1700);
'''

In [8]:
cur.executescript(sql_salaries)
con.commit()

In [9]:
pd.read_sql(
    """
SELECT *
FROM salaries
    """,
    con,
)

,emp_id,depname,emp,role,salary
0,1,dep1,Christiano,Manager,1000
1,2,dep1,Eric,Manager,1500
2,3,dep1,Ryan,Director,2000
3,4,dep1,Bryan,Director,1500
4,5,dep2,Doreen,Manager,2000
5,6,dep2,Gabriel,Manager,3000
6,7,dep2,Angel,Manager,2400
7,8,dep2,Robin,Director,2700
8,9,dep3,Sarah,Manager,1900
9,10,dep3,Peter,Manager,1700


In [11]:
pd.read_sql(
    """
SELECT avg(salary) as 'avg'
FROM salaries;
    """,
    con,
)

,avg
0,1970.0


In [12]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, avg(salary) OVER() as 'avg'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,avg
0,1,Christiano,dep1,1000,1970.0
1,2,Eric,dep1,1500,1970.0
2,3,Ryan,dep1,2000,1970.0
3,4,Bryan,dep1,1500,1970.0
4,5,Doreen,dep2,2000,1970.0
5,6,Gabriel,dep2,3000,1970.0
6,7,Angel,dep2,2400,1970.0
7,8,Robin,dep2,2700,1970.0
8,9,Sarah,dep3,1900,1970.0
9,10,Peter,dep3,1700,1970.0


In [13]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, abs(salary - avg(salary) OVER ()) as 'diff_with_avg'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,diff_with_avg
0,1,Christiano,dep1,1000,970.0
1,2,Eric,dep1,1500,470.0
2,3,Ryan,dep1,2000,30.0
3,4,Bryan,dep1,1500,470.0
4,5,Doreen,dep2,2000,30.0
5,6,Gabriel,dep2,3000,1030.0
6,7,Angel,dep2,2400,430.0
7,8,Robin,dep2,2700,730.0
8,9,Sarah,dep3,1900,70.0
9,10,Peter,dep3,1700,270.0


In [14]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, avg(salary) OVER (PARTITION BY depname) as 'avg_by_dept'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,avg_by_dept
0,1,Christiano,dep1,1000,1500.0
1,2,Eric,dep1,1500,1500.0
2,3,Ryan,dep1,2000,1500.0
3,4,Bryan,dep1,1500,1500.0
4,5,Doreen,dep2,2000,2525.0
5,6,Gabriel,dep2,3000,2525.0
6,7,Angel,dep2,2400,2525.0
7,8,Robin,dep2,2700,2525.0
8,9,Sarah,dep3,1900,1800.0
9,10,Peter,dep3,1700,1800.0


In [15]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, abs(salary - avg(salary) OVER (PARTITION BY depname))  as 'diff_with_avg_by_dept'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,diff_with_avg_by_dept
0,1,Christiano,dep1,1000,500.0
1,2,Eric,dep1,1500,0.0
2,3,Ryan,dep1,2000,500.0
3,4,Bryan,dep1,1500,0.0
4,5,Doreen,dep2,2000,525.0
5,6,Gabriel,dep2,3000,475.0
6,7,Angel,dep2,2400,125.0
7,8,Robin,dep2,2700,175.0
8,9,Sarah,dep3,1900,100.0
9,10,Peter,dep3,1700,100.0


In [16]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, sum(salary) OVER () as 'sum'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,sum
0,1,Christiano,dep1,1000,19700
1,2,Eric,dep1,1500,19700
2,3,Ryan,dep1,2000,19700
3,4,Bryan,dep1,1500,19700
4,5,Doreen,dep2,2000,19700
5,6,Gabriel,dep2,3000,19700
6,7,Angel,dep2,2400,19700
7,8,Robin,dep2,2700,19700
8,9,Sarah,dep3,1900,19700
9,10,Peter,dep3,1700,19700


In [17]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, salary*100/(sum(salary) OVER ()) as 'prc'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,prc
0,1,Christiano,dep1,1000,5
1,2,Eric,dep1,1500,7
2,3,Ryan,dep1,2000,10
3,4,Bryan,dep1,1500,7
4,5,Doreen,dep2,2000,10
5,6,Gabriel,dep2,3000,15
6,7,Angel,dep2,2400,12
7,8,Robin,dep2,2700,13
8,9,Sarah,dep3,1900,9
9,10,Peter,dep3,1700,8


In [18]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(salary*100/(sum(salary) OVER ()),1) as 'prc'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,prc
0,1,Christiano,dep1,1000,5.0
1,2,Eric,dep1,1500,7.0
2,3,Ryan,dep1,2000,10.0
3,4,Bryan,dep1,1500,7.0
4,5,Doreen,dep2,2000,10.0
5,6,Gabriel,dep2,3000,15.0
6,7,Angel,dep2,2400,12.0
7,8,Robin,dep2,2700,13.0
8,9,Sarah,dep3,1900,9.0
9,10,Peter,dep3,1700,8.0


In [19]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, cast(salary*100/(sum(salary) OVER ()) as float) as 'prc'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,prc
0,1,Christiano,dep1,1000,5.0
1,2,Eric,dep1,1500,7.0
2,3,Ryan,dep1,2000,10.0
3,4,Bryan,dep1,1500,7.0
4,5,Doreen,dep2,2000,10.0
5,6,Gabriel,dep2,3000,15.0
6,7,Angel,dep2,2400,12.0
7,8,Robin,dep2,2700,13.0
8,9,Sarah,dep3,1900,9.0
9,10,Peter,dep3,1700,8.0


In [20]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(cast(salary as float)*100/(sum(salary) OVER ()),2) as 'prc'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,prc
0,1,Christiano,dep1,1000,5.08
1,2,Eric,dep1,1500,7.61
2,3,Ryan,dep1,2000,10.15
3,4,Bryan,dep1,1500,7.61
4,5,Doreen,dep2,2000,10.15
5,6,Gabriel,dep2,3000,15.23
6,7,Angel,dep2,2400,12.18
7,8,Robin,dep2,2700,13.71
8,9,Sarah,dep3,1900,9.64
9,10,Peter,dep3,1700,8.63


In [21]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(salary*100/cast(sum(salary) OVER () as float),2) as 'prc'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,prc
0,1,Christiano,dep1,1000,5.08
1,2,Eric,dep1,1500,7.61
2,3,Ryan,dep1,2000,10.15
3,4,Bryan,dep1,1500,7.61
4,5,Doreen,dep2,2000,10.15
5,6,Gabriel,dep2,3000,15.23
6,7,Angel,dep2,2400,12.18
7,8,Robin,dep2,2700,13.71
8,9,Sarah,dep3,1900,9.64
9,10,Peter,dep3,1700,8.63


In [22]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(salary*100.0/(sum(salary) OVER ()),2) as 'prc'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,prc
0,1,Christiano,dep1,1000,5.08
1,2,Eric,dep1,1500,7.61
2,3,Ryan,dep1,2000,10.15
3,4,Bryan,dep1,1500,7.61
4,5,Doreen,dep2,2000,10.15
5,6,Gabriel,dep2,3000,15.23
6,7,Angel,dep2,2400,12.18
7,8,Robin,dep2,2700,13.71
8,9,Sarah,dep3,1900,9.64
9,10,Peter,dep3,1700,8.63


In [23]:
pd.read_sql(
    """
    
 select q.*, 
 sum(q.prc) OVER () as 'new_prc' 
 
 from 
 (
SELECT emp_id,
emp, --'сотрудник'
depname, 
salary, 
round(salary*1.0/(sum(salary) OVER ()),3) as 'prc'

FROM salaries) q

    """,
    con,
)

,emp_id,emp,depname,salary,prc,new_prc
0,1,Christiano,dep1,1000,0.051,1.0
1,2,Eric,dep1,1500,0.076,1.0
2,3,Ryan,dep1,2000,0.102,1.0
3,4,Bryan,dep1,1500,0.076,1.0
4,5,Doreen,dep2,2000,0.102,1.0
5,6,Gabriel,dep2,3000,0.152,1.0
6,7,Angel,dep2,2400,0.122,1.0
7,8,Robin,dep2,2700,0.137,1.0
8,9,Sarah,dep3,1900,0.096,1.0
9,10,Peter,dep3,1700,0.086,1.0


In [24]:
pd.read_sql(
    """
SELECT 1/2
    """,
    con,
)

,1/2
0,0


In [25]:
pd.read_sql(
    """
SELECT 1.0/2
    """,
    con,
)

,1.0/2
0,0.5


In [26]:
pd.read_sql(
    """
SELECT 1/2.0
    """,
    con,
)

,1/2.0
0,0.5


In [27]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, sum(salary) OVER (PARTITION BY depname) as 'sum'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,sum
0,1,Christiano,dep1,1000,6000
1,2,Eric,dep1,1500,6000
2,3,Ryan,dep1,2000,6000
3,4,Bryan,dep1,1500,6000
4,5,Doreen,dep2,2000,10100
5,6,Gabriel,dep2,3000,10100
6,7,Angel,dep2,2400,10100
7,8,Robin,dep2,2700,10100
8,9,Sarah,dep3,1900,3600
9,10,Peter,dep3,1700,3600


In [28]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, sum(salary) OVER (PARTITION BY depname ORDER BY salary) as 'sum'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,sum
0,1,Christiano,dep1,1000,1000
1,2,Eric,dep1,1500,4000
2,4,Bryan,dep1,1500,4000
3,3,Ryan,dep1,2000,6000
4,5,Doreen,dep2,2000,2000
5,7,Angel,dep2,2400,4400
6,8,Robin,dep2,2700,7100
7,6,Gabriel,dep2,3000,10100
8,10,Peter,dep3,1700,1700
9,9,Sarah,dep3,1900,3600


In [29]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary,
sum(salary) OVER (PARTITION BY depname ORDER BY salary ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING) as 'sum'
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,sum
0,1,Christiano,dep1,1000,2500
1,2,Eric,dep1,1500,3000
2,4,Bryan,dep1,1500,3500
3,3,Ryan,dep1,2000,2000
4,5,Doreen,dep2,2000,4400
5,7,Angel,dep2,2400,5100
6,8,Robin,dep2,2700,5700
7,6,Gabriel,dep2,3000,3000
8,10,Peter,dep3,1700,3600
9,9,Sarah,dep3,1900,1900


In [30]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, row_number() OVER (PARTITION BY depname) row_num_by_dept
FROM salaries;
    """,
    con,
)

,emp_id,emp,depname,salary,row_num_by_dept
0,1,Christiano,dep1,1000,1
1,2,Eric,dep1,1500,2
2,3,Ryan,dep1,2000,3
3,4,Bryan,dep1,1500,4
4,5,Doreen,dep2,2000,1
5,6,Gabriel,dep2,3000,2
6,7,Angel,dep2,2400,3
7,8,Robin,dep2,2700,4
8,9,Sarah,dep3,1900,1
9,10,Peter,dep3,1700,2


In [31]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,row_num_by_dept
0,3,Ryan,dep1,2000,1
1,2,Eric,dep1,1500,2
2,4,Bryan,dep1,1500,3
3,1,Christiano,dep1,1000,4
4,6,Gabriel,dep2,3000,1
5,8,Robin,dep2,2700,2
6,7,Angel,dep2,2400,3
7,5,Doreen,dep2,2000,4
8,9,Sarah,dep3,1900,1
9,10,Peter,dep3,1700,2


In [32]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, rank() OVER (PARTITION BY depname ORDER BY salary DESC) rank_by_dept
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,rank_by_dept
0,3,Ryan,dep1,2000,1
1,2,Eric,dep1,1500,2
2,4,Bryan,dep1,1500,2
3,1,Christiano,dep1,1000,4
4,6,Gabriel,dep2,3000,1
5,8,Robin,dep2,2700,2
6,7,Angel,dep2,2400,3
7,5,Doreen,dep2,2000,4
8,9,Sarah,dep3,1900,1
9,10,Peter,dep3,1700,2


In [33]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, dense_rank() OVER (PARTITION BY depname ORDER BY salary DESC) rank_by_dept
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,rank_by_dept
0,3,Ryan,dep1,2000,1
1,2,Eric,dep1,1500,2
2,4,Bryan,dep1,1500,2
3,1,Christiano,dep1,1000,3
4,6,Gabriel,dep2,3000,1
5,8,Robin,dep2,2700,2
6,7,Angel,dep2,2400,3
7,5,Doreen,dep2,2000,4
8,9,Sarah,dep3,1900,1
9,10,Peter,dep3,1700,2


In [34]:
pd.read_sql(
    """
select * from (
SELECT emp_id, emp,depname, salary, row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
) r where r.row_num_by_dept = 1
    """,
    con,
)

,emp_id,emp,depname,salary,row_num_by_dept
0,3,Ryan,dep1,2000,1
1,6,Gabriel,dep2,3000,1
2,9,Sarah,dep3,1900,1


In [35]:
pd.read_sql(
    """
with ranks as (
SELECT emp_id, emp,depname, salary,
row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
) 
select * from ranks where row_num_by_dept = 1
    """,
    con,
)

,emp_id,emp,depname,salary,row_num_by_dept
0,3,Ryan,dep1,2000,1
1,6,Gabriel,dep2,3000,1
2,9,Sarah,dep3,1900,1


In [36]:
pd.read_sql(
    """
with ranks as (
SELECT emp_id, emp,depname, salary,
row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
) 
select * from ranks where row_num_by_dept = 1
union all
select * from ranks where row_num_by_dept = 2
union all
select * from ranks where row_num_by_dept = 3
    """,
    con,
)

,emp_id,emp,depname,salary,row_num_by_dept
0,3,Ryan,dep1,2000,1
1,6,Gabriel,dep2,3000,1
2,9,Sarah,dep3,1900,1
3,2,Eric,dep1,1500,2
4,8,Robin,dep2,2700,2
5,10,Peter,dep3,1700,2
6,4,Bryan,dep1,1500,3
7,7,Angel,dep2,2400,3


In [37]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,NTILE(2) OVER(ORDER BY salary DESC) AS 'NTILE' 
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,NTILE
0,6,Gabriel,dep2,3000,1
1,8,Robin,dep2,2700,1
2,7,Angel,dep2,2400,1
3,3,Ryan,dep1,2000,1
4,5,Doreen,dep2,2000,1
5,9,Sarah,dep3,1900,2
6,10,Peter,dep3,1700,2
7,2,Eric,dep1,1500,2
8,4,Bryan,dep1,1500,2
9,1,Christiano,dep1,1000,2


In [38]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAG(salary) OVER(PARTITION BY depname ORDER BY salary DESC) AS 'LAG' 
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,LAG
0,3,Ryan,dep1,2000,NaN
1,2,Eric,dep1,1500,2000.0
2,4,Bryan,dep1,1500,1500.0
3,1,Christiano,dep1,1000,1500.0
4,6,Gabriel,dep2,3000,NaN
5,8,Robin,dep2,2700,3000.0
6,7,Angel,dep2,2400,2700.0
7,5,Doreen,dep2,2000,2400.0
8,9,Sarah,dep3,1900,NaN
9,10,Peter,dep3,1700,1900.0


In [39]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LEAD(salary) OVER(PARTITION BY depname ORDER BY salary DESC) AS 'LEAD' 
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,LEAD
0,3,Ryan,dep1,2000,1500.0
1,2,Eric,dep1,1500,1500.0
2,4,Bryan,dep1,1500,1000.0
3,1,Christiano,dep1,1000,NaN
4,6,Gabriel,dep2,3000,2700.0
5,8,Robin,dep2,2700,2400.0
6,7,Angel,dep2,2400,2000.0
7,5,Doreen,dep2,2000,NaN
8,9,Sarah,dep3,1900,1700.0
9,10,Peter,dep3,1700,NaN


In [40]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,FIRST_VALUE(salary) OVER(PARTITION BY depname) AS 'FIRST_VALUE' 
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,FIRST_VALUE
0,1,Christiano,dep1,1000,1000
1,2,Eric,dep1,1500,1000
2,3,Ryan,dep1,2000,1000
3,4,Bryan,dep1,1500,1000
4,5,Doreen,dep2,2000,2000
5,6,Gabriel,dep2,3000,2000
6,7,Angel,dep2,2400,2000
7,8,Robin,dep2,2700,2000
8,9,Sarah,dep3,1900,1900
9,10,Peter,dep3,1700,1900


In [41]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAST_VALUE(salary) OVER(PARTITION BY depname) AS 'LAST_VALUE' 
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,LAST_VALUE
0,1,Christiano,dep1,1000,1500
1,2,Eric,dep1,1500,1500
2,3,Ryan,dep1,2000,1500
3,4,Bryan,dep1,1500,1500
4,5,Doreen,dep2,2000,2700
5,6,Gabriel,dep2,3000,2700
6,7,Angel,dep2,2400,2700
7,8,Robin,dep2,2700,2700
8,9,Sarah,dep3,1900,1700
9,10,Peter,dep3,1700,1700


In [42]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAST_VALUE(salary) OVER(PARTITION BY depname ORDER BY salary) AS 'LAST_VALUE' 
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,LAST_VALUE
0,1,Christiano,dep1,1000,1000
1,2,Eric,dep1,1500,1500
2,4,Bryan,dep1,1500,1500
3,3,Ryan,dep1,2000,2000
4,5,Doreen,dep2,2000,2000
5,7,Angel,dep2,2400,2400
6,8,Robin,dep2,2700,2700
7,6,Gabriel,dep2,3000,3000
8,10,Peter,dep3,1700,1700
9,9,Sarah,dep3,1900,1900


In [43]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAST_VALUE(salary) OVER(PARTITION BY depname ORDER BY salary
ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
) AS 'LAST_VALUE' 
FROM salaries
    """,
    con,
)

,emp_id,emp,depname,salary,LAST_VALUE
0,1,Christiano,dep1,1000,2000
1,2,Eric,dep1,1500,2000
2,4,Bryan,dep1,1500,2000
3,3,Ryan,dep1,2000,2000
4,5,Doreen,dep2,2000,3000
5,7,Angel,dep2,2400,3000
6,8,Robin,dep2,2700,3000
7,6,Gabriel,dep2,3000,3000
8,10,Peter,dep3,1700,1900
9,9,Sarah,dep3,1900,1900


1. Какую часть зарплат от общей суммы получают менеждеры, а какую директора?

In [44]:
pd.read_sql(
    """

SELECT s.*, sum(salary) over() as 'sum', sum(s.salary) over(partition by s.role) as 'by_role' ,
 (sum(s.salary) over(partition by s.role) )*100.0/(sum(salary) over()) as 'prc_role'
FROM salaries s 

    """,
    con,
)

,emp_id,depname,emp,role,salary,sum,by_role,prc_role
0,3,dep1,Ryan,Director,2000,19700,6200,31.472081
1,4,dep1,Bryan,Director,1500,19700,6200,31.472081
2,8,dep2,Robin,Director,2700,19700,6200,31.472081
3,1,dep1,Christiano,Manager,1000,19700,13500,68.527919
4,2,dep1,Eric,Manager,1500,19700,13500,68.527919
5,5,dep2,Doreen,Manager,2000,19700,13500,68.527919
6,6,dep2,Gabriel,Manager,3000,19700,13500,68.527919
7,7,dep2,Angel,Manager,2400,19700,13500,68.527919
8,9,dep3,Sarah,Manager,1900,19700,13500,68.527919
9,10,dep3,Peter,Manager,1700,19700,13500,68.527919


2. На каком месте по величине зарплаты находятся сотрудники, у которых первая буква в имени "R"? 

In [45]:
pd.read_sql(
    """
select q.* from (
SELECT emp_id, emp,depname, salary, dense_rank() OVER ( ORDER BY salary DESC) rank_by_dept
FROM salaries
) q
where emp like 'R%'
    """,
    con,
)

,emp_id,emp,depname,salary,rank_by_dept
0,8,Robin,dep2,2700,2
1,3,Ryan,dep1,2000,4


3. Выведите процент менеджеров и директоров по каждому из департаментов.

In [46]:
pd.read_sql(
    """
    select q.depname,q.role,q.prc_man_dir from (
SELECT depname,role,salary,
--count() over(partition by depname,role ) as 'cnt_dep_role',count() over(partition by depname ) as 'cnt_dep',
(count() over(partition by depname,role ))*100.0 /count() over(partition by depname ) as 'prc_man_dir'
FROM salaries
) q
group by q.depname,q.role,q.prc_man_dir
    """,
    con,
)

,depname,role,prc_man_dir
0,dep1,Director,50.0
1,dep1,Manager,50.0
2,dep2,Director,25.0
3,dep2,Manager,75.0
4,dep3,Manager,100.0


In [47]:
pd.read_sql(
    """
SELECT depname,role
FROM salaries q
group by q.depname,q.role
    """,
    con,
)

,depname,role
0,dep1,Director
1,dep1,Manager
2,dep2,Director
3,dep2,Manager
4,dep3,Manager


4. Выведите среднюю зарплату по 1 департаменту, а также по 2 и 3 совместно.

In [48]:
pd.read_sql(
    """
SELECT emp_id,depname,emp,role,salary,
avg(salary) over (partition by depname)
FROM salaries
where depname = 'dep1'
union all
SELECT emp_id,depname,emp,role,salary
,avg(salary) over (partition by depname)
FROM salaries
where depname != 'dep1'
    """,
    con,
)

,emp_id,depname,emp,role,salary,avg(salary) over (partition by depname)
0,1,dep1,Christiano,Manager,1000,1500.0
1,2,dep1,Eric,Manager,1500,1500.0
2,3,dep1,Ryan,Director,2000,1500.0
3,4,dep1,Bryan,Director,1500,1500.0
4,5,dep2,Doreen,Manager,2000,2525.0
5,6,dep2,Gabriel,Manager,3000,2525.0
6,7,dep2,Angel,Manager,2400,2525.0
7,8,dep2,Robin,Director,2700,2525.0
8,9,dep3,Sarah,Manager,1900,1800.0
9,10,dep3,Peter,Manager,1700,1800.0


In [49]:
pd.read_sql(
    """
    with  avg_sal as (
SELECT distinct depname,
avg(salary) over (partition by depname) as 'avg_'
FROM salaries
)
select * from avg_sal where depname = 'dep1'
union all
select 'dep2+dep3', sum(avg_) from avg_sal where depname != 'dep1'

    """,
    con,
)

,depname,avg_
0,dep1,1500.0
1,dep2+dep3,4325.0


5. Какой процент от суммы зарплат по всех организации получают 10% сотрудников с самыми большими зарплатами? 20%?30%?

In [50]:
pd.read_sql(
    """
  with table1 as (  
SELECT emp_id,depname,emp,role,salary,
sum(salary) over () as 'sum_all',
count() over () as 'cnt_all',
count() over (order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND current row) as 'sum_emp',
sum(salary) over (order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND current row) as 'sum_sal'
FROM salaries
)

select sum_emp*100.0/cnt_all as 'true_prc',
table1.salary,table1.sum_all,table1.sum_sal,
sum_sal*100.0/sum_all


from table1
    """,
    con,
)

,true_prc,salary,sum_all,sum_sal,sum_sal*100.0/sum_all
0,10.0,3000,19700,3000,15.228426
1,20.0,2700,19700,5700,28.934010
2,30.0,2400,19700,8100,41.116751
3,40.0,2000,19700,10100,51.269036
4,50.0,2000,19700,12100,61.421320
5,60.0,1900,19700,14000,71.065990
6,70.0,1700,19700,15700,79.695431
7,80.0,1500,19700,17200,87.309645
8,90.0,1500,19700,18700,94.923858
9,100.0,1000,19700,19700,100.000000


In [51]:
pd.read_sql(
    """
  with table1 as (  
SELECT emp_id,depname,emp,role,salary,
sum(salary) over () as 'sum_all',
--count() over () as 'cnt_all',
count() over (order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND current row) as 'sum_emp',
NTILE(2) OVER(order by salary)
FROM salaries
)

select *  from table1
    """,
    con,
)

,emp_id,depname,emp,role,salary,sum_all,sum_emp,NTILE(2) OVER(order by salary)
0,6,dep2,Gabriel,Manager,3000,19700,1,2
1,8,dep2,Robin,Director,2700,19700,2,2
2,7,dep2,Angel,Manager,2400,19700,3,2
3,3,dep1,Ryan,Director,2000,19700,4,2
4,5,dep2,Doreen,Manager,2000,19700,5,2
5,9,dep3,Sarah,Manager,1900,19700,6,1
6,10,dep3,Peter,Manager,1700,19700,7,1
7,2,dep1,Eric,Manager,1500,19700,8,1
8,4,dep1,Bryan,Director,1500,19700,9,1
9,1,dep1,Christiano,Manager,1000,19700,10,1
